# Sample Database Code

Steal this code.

The TIdatabase module encapsulates all of the loading, storing and joining of the student, college and applications.

The module should be imported at the beginning of your code, whether it is a regular Python file or an iPython Notebook.

In [1]:
import TIdatabase as ti
import pandas as pd

## Generating Mock Data

In [2]:

students = ti.Student()
# populate students with random values
students.fillRandom(10)
students.df 

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,22UJ5OP5CY,0.091667,0.160120,0.134838,0.352085,0.276764,0.501544,2,1,2013,...,0,1,1,1,0,1,0,1,0,0
1,THWA8ERSMO,0.724631,0.832784,0.001853,0.800325,0.507890,0.039386,2,2,2017,...,0,1,0,1,0,0,1,1,0,1
2,YAZWBBWAMB,0.227129,0.819268,0.838481,0.479735,0.029056,0.850585,2,3,2012,...,0,0,0,0,0,0,0,1,1,1
3,3LF5LS7BP7,0.180070,0.644786,0.892562,0.718931,0.860014,0.262924,2,5,2011,...,1,0,1,0,1,1,0,1,1,0
4,2RF4YKEUSB,0.374885,0.251988,0.826283,0.946828,0.650145,0.041863,3,4,2020,...,0,1,0,1,1,0,1,1,0,1
5,COKVLM9UJC,0.342895,0.184984,0.197940,0.735857,0.549010,0.021939,5,1,2012,...,1,1,1,0,1,0,1,1,0,1
6,TOZX9VZOK2,0.248635,0.484192,0.562855,0.555474,0.406336,0.164143,5,5,2013,...,1,1,1,0,1,1,0,1,1,0
7,3WD8CUXVIE,0.684983,0.707700,0.650911,0.984126,0.291441,0.499700,2,4,2011,...,1,1,0,1,1,0,1,1,1,1
8,EN9W33AMAX,0.913737,0.405216,0.650422,0.765078,0.835190,0.832098,5,4,2010,...,1,0,1,0,0,0,0,0,0,0
9,AK3XSE85Q2,0.784712,0.482233,0.878633,0.234746,0.882323,0.865981,5,2,2016,...,0,0,0,1,1,1,1,1,0,0


The list of colleges is populated from a CSV stored in the same directory as this notebook. It can be edited using your favorite CSV editor, such as Excel. When you create a new instance of College, the values will be read in from the CSV

In [3]:
# populate with list of known colleges
colleges = ti.College()
colleges.df

,collegeID,name,acceptrate,size,public,finAidPct,instatePct
0,Princeton,Princeton,7.40%,5142,-1,0.00,0
1,Harvard,Harvard,6.00%,19929,-1,0.75,0
2,Yale,Yale,6.30%,12336,-1,0.00,0
3,Columbia,Columbia,7.00%,24221,-1,0.00,0
4,Stanford,Stanford,5.10%,16795,-1,0.00,0
5,UChicago,UChicago,8.80%,12558,-1,0.00,0
6,MIT,MIT,7.90%,11319,-1,0.00,0
7,Duke,Duke,11.40%,15856,-1,0.00,0
8,UPenn,UPenn,10.40%,21296,-1,0.00,0
9,CalTech,CalTech,8.80%,2209,-1,0.00,0


This is the table of application forms. "acceptStatus" is our predictor. "acceptProb" is our $\hat{Y}$ probability.

In [4]:
applForm = ti.ApplForm()
applForm.fillRandom(30)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,YAZWBBWAMB,Northwestern,1,1,0,0.213352
1,2RF4YKEUSB,CalTech,0,1,0,0.259748
2,AK3XSE85Q2,Stanford,1,1,0,0.812788
3,EN9W33AMAX,Harvard,1,1,1,0.705758
4,EN9W33AMAX,UPenn,0,0,1,0.913792
5,3WD8CUXVIE,USC,1,1,0,0.045028
6,TOZX9VZOK2,Northwestern,0,0,0,0.376894
7,2RF4YKEUSB,MIT,1,1,1,0.784409
8,EN9W33AMAX,Princeton,0,1,0,0.831893
9,3WD8CUXVIE,Stanford,0,0,0,0.721269


To combine the student and application forms tables, use Pandas merge capability, which will match rows by identical column names, which is studentID in this case:

In [5]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,firstinfamily,alumni,sports,artist,workexp,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,22UJ5OP5CY,0.091667,0.160120,0.134838,0.352085,0.276764,0.501544,2,1,2013,...,1,0,1,0,0,Northwestern,1,0,1,0.820636
1,22UJ5OP5CY,0.091667,0.160120,0.134838,0.352085,0.276764,0.501544,2,1,2013,...,1,0,1,0,0,Emory,1,0,0,0.716759
2,THWA8ERSMO,0.724631,0.832784,0.001853,0.800325,0.507890,0.039386,2,2,2017,...,0,1,1,0,1,MIT,0,1,1,0.829099
3,THWA8ERSMO,0.724631,0.832784,0.001853,0.800325,0.507890,0.039386,2,2,2017,...,0,1,1,0,1,Emory,1,0,0,0.328038
4,THWA8ERSMO,0.724631,0.832784,0.001853,0.800325,0.507890,0.039386,2,2,2017,...,0,1,1,0,1,JohnsHopkins,0,0,0,0.501368
5,YAZWBBWAMB,0.227129,0.819268,0.838481,0.479735,0.029056,0.850585,2,3,2012,...,0,0,1,1,1,Northwestern,1,1,0,0.213352
6,YAZWBBWAMB,0.227129,0.819268,0.838481,0.479735,0.029056,0.850585,2,3,2012,...,0,0,1,1,1,Emory,0,0,1,0.730900
7,3LF5LS7BP7,0.180070,0.644786,0.892562,0.718931,0.860014,0.262924,2,5,2011,...,1,0,1,1,0,UCB,0,0,0,0.485450
8,2RF4YKEUSB,0.374885,0.251988,0.826283,0.946828,0.650145,0.041863,3,4,2020,...,0,1,1,0,1,CalTech,0,1,0,0.259748
9,2RF4YKEUSB,0.374885,0.251988,0.826283,0.946828,0.650145,0.041863,3,4,2020,...,0,1,1,0,1,MIT,1,1,1,0.784409


Now the `applications` Pandas DataFrame is ready to use for either regression (by overwriting the acceptProb column) or building the public facing web site.

## Saving Scraped Data

### Part 1 - The Student Data

First, let's start fresh and delete the previously created objects. This is only necessary because this sample script is running within Jupyter where all variables are global to the page. In a separate Python file run from the command line, this step can be skipped. Creating a new instance would not work as behind the scenes, there is sharing of
DataFrames between objects.

In [6]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm


Let's create a new students instance. It will be an empty Pandas dataframe with the correct columns

In [7]:
students = ti.Student()
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp


Populate a dictionary with the values that the scraper has for a given row. Make sure the keys match up with the column names as only the matching columns will be saved. There is no need to add the studentID key. A unique value will be generated automatically and returned from the insert. This will be in the same order as the provided rows. Saving the generated student IDs will be helpful later when populating the applForm foreign key.

In [8]:

# Example: international male who scored in 98th percentile in ACT/SAT, went to a public school and is applying for
# Class of 2020

newrow = {'admissionstest': 0.98,
         'GPA': 4.5,
         'female' : -1,
         'international': 1,
         'schooltype': -1,
         'intendedgradyear':2020}

newstudentIDs = students.insert(newrow)
print "New studentID:",newstudentIDs
students.df

New studentID: ['9LIIBZ55QP']


,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,9LIIBZ55QP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


It is more efficient if multiple rows are added in one step. In this case, create a list of dictionaries and just
use the same method. Here, two new rows are added to the DataFrame in one step.

In [9]:
rows = []
a = {'schooltype': -1, 'admissionstest': 0.98, 'GPA': 4.5, 'female': 1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
a = {'schooltype': 1, 'admissionstest': 0.65, 'GPA': 2.2, 'female': -1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
newstudentIDs = students.insert(rows)
print "New studentIDs:",newstudentIDs
students.df

New studentIDs: ['RNW20W43PX', 'V43U7DSXKD']


,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,9LIIBZ55QP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,RNW20W43PX,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,V43U7DSXKD,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,-1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Now we are ready to save. The data is saved in CSV format for ease of interpretability.

In [10]:
students.save("mydata.csv")

Let's delete all the data and check that we can read it back successfully.

In [11]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm



In [12]:
students = ti.Student()
students.read("mydata.csv")
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,9LIIBZ55QP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,RNW20W43PX,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,V43U7DSXKD,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,-1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Et, voilà, the data is back.

### Part 2 - The Application Data

This is pretty much the same, **except** for two important differences:

* The studentID and collegeID must both be populated and exist in the respective DataFrames
* The columns are a little different. Normally acceptProb would not be populated from the scraper but could be used to store prediction runs. 

In [13]:
# we already wiped out applForm above
applForm = ti.ApplForm()
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb


In [14]:
studentID = students.df.iloc[1].studentID
collegeID = colleges.df.iloc[1].collegeID

In [15]:
print studentID, collegeID

RNW20W43PX Harvard


In [16]:
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}

applForm.insert(newrow)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RNW20W43PX,Harvard,0,0,0,NaN


Now inserting multiple rows

In [17]:
rows = []
studentID = students.df.iloc[2].studentID
collegeID = colleges.df.iloc[2].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}
rows.append(newrow)
# note: same student, new school to apply to
studentID = students.df.iloc[2].studentID
collegeID = colleges.df.iloc[3].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 1,
         'visited': 1,
         'acceptStatus': 1}
rows.append(newrow)
applForm.insert(rows)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RNW20W43PX,Harvard,0,0,0,NaN
0,V43U7DSXKD,Yale,0,0,0,NaN
1,V43U7DSXKD,Columbia,1,1,1,NaN


Let's save it

In [18]:
applForm.save("applform1.csv")

Then delete the local variable

In [19]:
if ('applForm' in locals()): del applForm

Then read it back

In [20]:
applForm = ti.ApplForm()
applForm.read("applform1.csv")
applForm.df


,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RNW20W43PX,Harvard,0,0,0,NaN
0,V43U7DSXKD,Yale,0,0,0,NaN
1,V43U7DSXKD,Columbia,1,1,1,NaN


And now let's check that the merge still works

In [21]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,firstinfamily,alumni,sports,artist,workexp,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RNW20W43PX,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,NaN,NaN,NaN,NaN,NaN,Harvard,0,0,0,NaN
1,V43U7DSXKD,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,NaN,NaN,NaN,NaN,NaN,Yale,0,0,0,NaN
2,V43U7DSXKD,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,NaN,NaN,NaN,NaN,NaN,Columbia,1,1,1,NaN


Ok, we are done for today.